# SQL Server 2019 Data Virtualization - Using Polybase to query HDFS (without Big Data Clusters)
This notebook contains an example of how to use external tables to query data in HDFS (not using Big Data Clusters) without moving data. You may need to change identity, secret, connection, database, schema, and remote table names to work with your HDFS system. This example uses Azure Blob Storage for HDFS.

This notebook also assumes you are using SQL Server 2019 Release Candidate or later and that the Polybase feature has been installed and enabled (you must choose the Java option when installing the Polybase feature to use external tables based on HDFS).

This notebook uses the sample WideWorldImporters sample database but can be used with any user database.

## Step 0: Create the storage for HDFS using Azure Blob Storage
Create an Azure Storage container to hold the hdfs data. For this example the name of my container is **wwi**. For further information look at the documentation at https://docs.microsoft.com/en-us/sql/relational-databases/polybase/polybase-configure-azure-blob-storage.

## Step 1: Enable Polybase connectivity to Azure Blog Storage and ingestion into HDFS
**You must restart SQL Server to be able to proceed to Step 2**

In [1]:
USE [master]
GO
sp_configure @configname = 'hadoop connectivity', @configvalue = 7;
GO
sp_configure 'allow polybase export', 1
GO
RECONFIGURE
GO

Commands completed successfully.

Configuration option 'hadoop connectivity' changed from 7 to 7. Run the RECONFIGURE statement to install.

Configuration option 'allow polybase export' changed from 1 to 1. Run the RECONFIGURE statement to install.

Commands completed successfully.

Total execution time: 00:00:00.046

## Step 2: Create a master key
Create a master key to encrypt the database credential

In [2]:
USE [WideWorldImporters]
GO
CREATE MASTER KEY ENCRYPTION BY PASSWORD = '<password>'
GO

Commands completed successfully.

Commands completed successfully.

Total execution time: 00:00:00.019

## Step 3: Create a database credential
Create a database scoped credential for access to Azure Blob Storage<br>
IDENTITY: any string (this is not used for authentication to Azure storage)<br>
SECRET: your Azure storage account key which you can get from the portal or az CLI

In [6]:
CREATE DATABASE SCOPED CREDENTIAL AzureStorageCredentials   
WITH IDENTITY = 'user', Secret = '<storage account key>'
GO

Commands completed successfully.

Total execution time: 00:00:00.004

## Step 4: Create an EXTERNAL DATA SOURCE
The EXTERNAL DATA SOURCE indicates what type of data source, the connection "string", where PUSHDOWN predicates should be used (if possible), and the name of the database credential.

For HDFS exteranl tables, you need to use the TYPE = HADOOP syntax. The LOCATION for Azure Blob Storage is the WASBS URI which you can get from the Azure Portal or az CLI

In [7]:
CREATE EXTERNAL DATA SOURCE bwdatalake with (  
      TYPE = HADOOP,
      LOCATION ='wasbs://wwi@<storage account>.blob.core.windows.net',  
      CREDENTIAL = AzureStorageCredentials
)
GO

Commands completed successfully.

Total execution time: 00:00:00.020

## Step 5: Create a file format for the external table
Use an EXTERNAL FILE FORMAT to define the format of the file in HDFS<br>
FORMAT TYPE: Type of format in Hadoop (DELIMITEDTEXT,  RCFILE, ORC, PARQUET).

In [9]:
CREATE EXTERNAL FILE FORMAT TextFileFormat WITH (  
      FORMAT_TYPE = DELIMITEDTEXT,
      FORMAT_OPTIONS (FIELD_TERMINATOR ='|',
            USE_TYPE_DEFAULT = TRUE))
GO

Commands completed successfully.

Total execution time: 00:00:00.021

## Step 6: Create a schema for the EXTERNAL TABLE
Schemas provide convenient methods to secure and organize objects

In [10]:
CREATE SCHEMA hdfs
GO

Commands completed successfully.

Total execution time: 00:00:00.004

## Step 7: Create an EXTERNAL TABLE
An external table provides metadata so SQL Server knows how to map columns to the remote table. In the case of HDFS, you can use any table or column name and map in appropriate data types for data in the file.

Create the external table to match the fileformat and file in HDFS<br>
LOCATION: path to file or directory that contains the data (relative to HDFS root).

In [19]:
CREATE EXTERNAL TABLE [hdfs].[WWI_Order_Reviews] (  
      [OrderID] int NOT NULL,
      [CustomerID] int NOT NULL,
      [Rating] int NULL,
      [Review_Comments] nvarchar(1000) NOT NULL
)  
WITH (LOCATION='/WWI/',
      DATA_SOURCE = bwdatalake,  
      FILE_FORMAT = TextFileFormat  
)
GO

Commands completed successfully.

Total execution time: 00:00:00.139

## Step 8: Ingest some data into HDFS
INSERT is allowed for external tables based on HDFS<br>
Ingest some data lined up with a valid OrderID and CustomerID in the database

In [20]:
INSERT INTO [hdfs].[WWI_Order_Reviews] VALUES (1, 832, 10, 'I had a great experience with my order')
GO

(1 row affected)

Total execution time: 00:00:00.689

## Step 9: Create statistics
SQL Server allows you to store local statistics about specific columns from the remote table. This can help the query processing to make more efficient plan decisions.

In [21]:
CREATE STATISTICS StatsforReviews on [hdfs].[WWI_Order_Reviews](OrderID, CustomerID)
GO

Commands completed successfully.

Total execution time: 00:00:00.491

## Step 10: Try to scan the remote table
Run a simple query on the EXTERNAL TABLE to scan all rows.

In [22]:
SELECT * FROM [hdfs].[WWI_Order_Reviews]
GO

(1 row affected)

Total execution time: 00:00:00.366

OrderID,CustomerID,Rating,Review_Comments
1,832,10,I had a great experience with my order


## Step 11: Query the remote table with a WHERE clause
Even though the table may be small SQL Server will "push" the WHERE clause filter to the remote table

In [23]:
SELECT * FROM [hdfs].[WWI_Order_Reviews]
WHERE OrderID = 1
GO

(1 row affected)

Total execution time: 00:00:00.572

OrderID,CustomerID,Rating,Review_Comments
1,832,10,I had a great experience with my order


## Step 12: Join with local SQL Server tables
Let's join the review with our order and customer data

In [24]:
SELECT o.OrderDate, c.CustomerName, p.FullName as SalesPerson, wor.Rating, wor.Review_Comments
FROM [Sales].[Orders] o
JOIN [hdfs].[WWI_Order_Reviews] wor
ON o.OrderID = wor.OrderID
JOIN [Application].[People] p
ON p.PersonID = o.SalespersonPersonID
JOIN [Sales].[Customers] c
ON c.CustomerID = wor.CustomerID
GO

(1 row affected)

Total execution time: 00:00:00.504

OrderDate,CustomerName,SalesPerson,Rating,Review_Comments
2013-01-01,Aakriti Byrraju,Kayla Woodcock,10,I had a great experience with my order
